In [1]:
# Import libraries
import psycopg2
import getpass
import numpy as np
import os.path
import datetime

In [2]:
# Create a database connection
user = 'postgres'
host = 'localhost'
port = '5433'
dbname = 'mimic'
schema = 'mimiciii'

In [3]:
# Connect to the database
con = psycopg2.connect(dbname=dbname, user=user, host=host, port=port, 
                       password=getpass.getpass(prompt='Password:'.format(user)))
cur = con.cursor()
cur.execute('SET search_path to {}'.format(schema))
con.commit()
print("Connected!")

Password: ··········


Connected!


In [4]:
# Load admission_ids
_adm = np.load('res/admission_ids.npy', allow_pickle=True).tolist()
admission_ids = _adm['admission_ids']

# Load item_ids
db = np.load('res/item_ids.npy', allow_pickle=True).tolist()
input_itemid = db['input']
output_itemid = db['output']
chart_itemid = db['chart']
lab_itemid = db['lab']
microbio_itemid = db['microbio']
prescript_itemid = db['prescript']

print("Loaded!")

Loaded!


In [5]:
# Query the database for the units of all item IDs, and how many entries use each unit
start = datetime.datetime.now()
lab_itemid_units = []
for i in lab_itemid[:10]:
    sql = \
    f'''
    SELECT valueuom, COUNT(*)
    FROM labevents
    WHERE itemid={i} AND hadm_id IN (
        SELECT *
        FROM admission_ids)
    GROUP BY valueuom
    '''
    cur.execute(sql)
    res = cur.fetchall()
    lab_itemid_units.append(res)
end = datetime.datetime.now()

In [6]:
# Check everything is working
print(end-start)
for i in range(10):
    print(lab_itemid_units[i])

0:00:01.194804
[(None, 197)]
[(None, 52)]
[('%', 26)]
[(None, 463)]
[('mg/dL', 9)]
[('mg/dL', 3378)]
[(None, 2)]
[(None, 1), ('ug/mL', 863)]
[(None, 1)]
[('%', 9)]
